# Regression: Predicting Pipe Pressure Drop

This notebook demonstrates supervised regression applied to a **CFD surrogate modeling** problem:
predicting fluid pressure drop across a pipe from flow parameters (flow rate, density, viscosity, length, diameter).
We generate synthetic data, fit a linear regression model, and visualize predictions vs. ground truth.

## Physics Problem: Pressure Drop in a Pipe

### Problem Illustration

```
                    Inlet                              Outlet
                     P₁                                   P₂
                     │                                    │
        ┌────────────▼────────────────────────────────────▼─────────┐
        │                                                             │
        │  ╔═══════════════════════════════════════════════════════╗ │
        │  ║        Pipe with flowing fluid                       ║ │
        │  ║  Q: flow rate (m³/s)                                 ║ │
        │  ║  ρ: density (kg/m³)                                  ║ │
        │  ║  μ: viscosity (Pa·s)                                 ║ │
        │  ║  L: pipe length (m)                                  ║ │
        │  ║  D: pipe diameter (m)                                ║ │
        │  ╚═══════════════════════════════════════════════════════╝ │
        │                                                             │
        └─────────────────────────────────────────────────────────────┘

              Δp = P₁ - P₂ (pressure drop)
```

### Physics Concept
When fluid flows through a pipe, friction and inertial forces cause a **pressure drop** (Δp) between inlet and outlet. This is governed by the **Darcy–Weisbach equation**:

$$\Delta p = \frac{1}{2} \rho v^2 \frac{L}{D} + \text{(viscous corrections)}$$

Where:
- **ρ**: fluid density (affects pressure magnitude)
- **v**: flow velocity proportional to Q/A (kinetic energy term)
- **L/D**: friction factor proxy (longer pipes, smaller diameters → higher loss)
- **μ**: viscosity (damping effect on flow)

### Machine Learning Goal
Given measurements of **Q, ρ, μ, L, D** (features), predict **Δp** (target) using linear regression.
The model learns the implicit coefficients that relate these features to pressure drop.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

plt.rcParams['figure.figsize'] = (6, 4)

In [ ]:
def compute_pressure_drop(flow_rate, density, viscosity, length, diameter):
    area = np.pi * (diameter / 2) ** 2
    velocity = flow_rate / area
    dp = 0.5 * density * velocity ** 2 * (length / diameter)
    dp += viscosity * flow_rate / (diameter ** 2)
    return dp

rng = np.random.RandomState(0)
n = 500
flow_rate = rng.uniform(0.01, 0.5, size=n)
density = rng.uniform(950.0, 1050.0, size=n)
viscosity = rng.uniform(1e-4, 5e-3, size=n)
length = rng.uniform(0.5, 10.0, size=n)
diameter = rng.uniform(0.01, 0.2, size=n)
X = np.vstack([flow_rate, density, viscosity, length, diameter]).T
y = np.array([compute_pressure_drop(q, rho, mu, L, D) for q, rho, mu, L, D in X])
y += rng.normal(scale=0.05 * np.mean(y), size=y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model = LinearRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Linear regression MSE: {mse:.4e}')
print('Coefficients:', model.coef_)

In [ ]:
plt.scatter(y_test, y_pred, alpha=0.6)

plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.xlabel('True pressure drop')
plt.ylabel('Predicted pressure drop')
plt.title('Predicted vs True (pipe pressure drop)')
plt.show()